In [15]:
## Importing libraries

import os
import random
import sys
import glob
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats
import warnings
from sklearn.model_selection import StratifiedShuffleSplit
from scipy import signal
from scipy.io import wavfile
from tqdm import tqdm

In [16]:
## Loading Data
def metadata(basepath):
    dir_list = os.listdir(basepath)
    dir_list.sort()

    ## DataFrame to save metadata of Ravdess audio files
    df = pd.DataFrame(columns=['path', 'source', 'actor', 'gender', 'intensity', 'statement', 'repetition', 'emotion'])
    count = 0

    ## Iterating through the directory, reading audio files, and extracting information from the file name
    for f in os.listdir(basepath):
        filename = f.split('.')[0].split('-')
        if(len(filename)==7):
            path = basepath + f
            src = int(filename[1])
            actor = int(filename[-1])
            emotion = int(filename[2])
            if int(actor)%2 == 0:
                gender = "female"
            else:
                gender = "male"

            if filename[3] == '01':
                intensity = 0
            else:
                intensity = 1

            if filename[4] == '01':
                statement = 0
            else:
                statement = 1

            if filename[5] == '01':
                repeat = 0
            else:
                repeat = 1

        df.loc[count] = [path, src, actor, gender, intensity, statement, repeat, emotion]
        count += 1

    labels = []
    for i in range(len(df)):
        if df.emotion[i] == 1:
            label = "_neutral"
        elif df.emotion[i] == 2:
            label = "_calm"
        elif df.emotion[i] == 3:
            label = "_happy"
        elif df.emotion[i] == 4:
            label = "_sad"
        elif df.emotion[i] == 5:
            label = "_angry"
        elif df.emotion[i] == 6:
            label = "_fearful"
        elif df.emotion[i] == 7:
            label = "_disgust"
        elif df.emotion[i] == 8:
            label = "_surprised"
        else:
            label = "_none"

        # Add gender to the label 
        labels.append(df.loc[i,'gender'] + label)

    df['label'] = labels
    
    return df

In [17]:
df = metadata("RAVDESS/")

In [18]:
df.head()

,path,source,actor,gender,intensity,statement,repetition,emotion,label
0,RAVDESS/03-01-08-01-01-01-19.wav,1,19,male,0,0,0,8,male_surprised
1,RAVDESS/03-01-08-01-01-01-23.wav,1,23,male,0,0,0,8,male_surprised
2,RAVDESS/03-01-02-01-02-01-12.wav,1,12,female,0,1,0,2,female_calm
3,RAVDESS/03-01-08-01-02-02-23.wav,1,23,male,0,1,1,8,male_surprised
4,RAVDESS/03-01-03-01-02-01-07.wav,1,7,male,0,1,0,3,male_happy


In [19]:
## Creating training and test dataframes through stratified sampling, test dataframes size 0.2*(total data)
sss = StratifiedShuffleSplit(n_splits=2, random_state=11, test_size=0.2)
for train_index, test_index in sss.split(df, df.label):
    df_train, df_test = df.iloc[train_index,:], df.iloc[test_index,:]

In [20]:
## Get features method takes in the metadata dataframe and spits out a dataframe with mfcc, mel scale and chroma features (180 features in total). 
def get_features(df):
    data = pd.DataFrame(columns=['feature'])
    label = pd.DataFrame(columns=['label'])
    name = pd.DataFrame(columns=['name'])
    paths = pd.DataFrame(columns=['path'])

    # for i, i_path in tqdm(zip(range(df.shape[0]), df['path'])):
    for index, row in tqdm(df.iterrows()):
      x, sample_rate = librosa.load(row['path'])

      ## Numpy array that will store all the features
      result=np.array([])

      ## MFCCs
      mfccs=np.mean(librosa.feature.mfcc(y=x, sr=sample_rate, n_mfcc=40).T, axis=0)
      result=np.hstack((result, mfccs))

      ## Chroma
      stft=np.abs(librosa.stft(x))
      chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
      result=np.hstack((result, chroma))

      ## Mel Scale
      mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
      result=np.hstack((result, mel))

      label.at[row['path'], 'label'] = row['label']
      data.at[row['path']] = [result]
      name.at[row['path'],'name'] = row['path'].split('/')[-1]
      paths.at[index, 'path'] = row['path']
        
    final_data = pd.DataFrame(data['feature'].values.tolist())
    final_data['label'] = label['label'].values.tolist()
    final_data['name'] = name['name'].values.tolist()
    final_data['path'] = paths['path'].values.tolist()
    final_data.set_index('path', inplace=True)

    return final_data

In [21]:
train_data = get_features(df_train)

0it [00:00, ?it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 0.000000e+00  0.000000e+00  0.000000e+00 ...  4.733409e-06 -5.749842e-07
  0.000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
1it [00:00,  1.69it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 2.9186014e-09 -4.2304968e-09  5.6335332e-09 ... -4.2903477e-05
 -5.7215530e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
2it [00:01,  2.03it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[-9.90880508e-05 -2.15280506e-05  3.06304864e-04 ... -5.35094186e-05
 -1.15133575e-04  0.00000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.m

24it [00:11,  2.14it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[-1.0150754e-05 -2.3243743e-05  6.0968332e-06 ...  4.0090832e-05
  4.2610081e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
25it [00:11,  2.18it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[-2.2186589e-06 -1.2793002e-05 -9.9140919e-08 ... -2.1504989e-05
  7.5506810e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
26it [00:12,  2.19it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
27it [00:12,  2.14it/s]

49it [00:23,  2.03it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[9.51426409e-05 1.64977813e-04 1.19230936e-04 ... 0.00000000e+00
 0.00000000e+00 0.00000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
50it [00:23,  2.09it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 4.4965947e-07 -1.0735834e-07 -9.1932020e-07 ... -1.1503717e-04
 -1.3214057e-04  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
51it [00:24,  2.15it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[2.2192053e-05 6.3646519e-05 4.9351100e-05 ... 3.0573767e-05 3.7237354e-05
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.

73it [00:34,  2.13it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -3.1233978e-07
  4.2266819e-07  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
74it [00:34,  2.16it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 2.0417898e-05  5.9584192e-05  2.2124757e-05 ... -4.9447781e-06
 -1.4294074e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
75it [00:35,  2.06it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  6.2299987e-06
 -3.3365050e-06  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.fe

98it [00:45,  2.19it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
99it [00:46,  2.15it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -1.8036471e-07
  1.5005152e-07  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
100it [00:46,  2.09it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 3.4688014e-06 -4.4427879e-06  6.5306685e-06 ...  3.9655160e-07
 -4.4222625e-07  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
101it [00:47,  2.09it/

...

1042it [08:14,  2.23it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
1043it [08:14,  2.20it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 2.1163190e-05  3.3689601e-05  2.8588782e-05 ... -3.1679455e-04
 -3.7560239e-04  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
1044it [08:15,  2.14it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[-1.4584244e-05  5.8123720e-07 -4.2101811e-07 ... -1.2112882e-06
  1.2402113e-06  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
1045it [08:15,  2

1066it [08:25,  2.10it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 3.5073111e-08 -2.4799833e-08  7.7988220e-09 ...  0.0000000e+00
  0.0000000e+00  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
1067it [08:25,  2.17it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 4.3596516e-05 5.8887922e-06
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
1068it [08:26,  2.16it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[-5.5930996e-05 -4.5582834e-05  6.3627931e-06 ...  0.0000000e+00
  0.0000000e+00  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.fe

1090it [08:36,  2.22it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 1.9022814e-06 -3.3974752e-06 -1.2565045e-05 ...  1.2024026e-05
 -1.0076522e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
1091it [08:37,  2.19it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  2.7588493e-08
 -1.3167187e-08  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
1092it [08:37,  2.19it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
1093it [08:37,  2

1114it [08:47,  2.16it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[-3.1201966e-05 -4.7064885e-05 -4.9508708e-05 ...  0.0000000e+00
  0.0000000e+00  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
1115it [08:48,  2.14it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
1116it [08:48,  2.18it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[-4.9541338e-04 -7.2310574e-04 -6.5073598e-04 ...  7.2543415e-05
  4.3088279e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
1117it [08:48,  2

1138it [08:58,  2.19it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[-2.2590991e-04 -3.0514476e-04 -6.9458445e-05 ...  1.9869907e-04
  2.1385770e-04  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
1139it [08:58,  2.18it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[4.5209417e-05 6.2389452e-05 6.6704393e-05 ... 6.0152710e-05 5.0643856e-05
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
1140it [08:59,  2.18it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[-7.4242216e-08 -1.6176133e-05 -1.9930822e-05 ...  1.9887529e-04
  1.8643003e-04  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.fe

In [22]:
test_data = get_features(df_test)

0it [00:00, ?it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
1it [00:00,  2.38it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[8.7517154e-05 1.3253895e-04 8.7894499e-05 ... 1.7546264e-04 1.9972424e-04
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
2it [00:00,  2.38it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 0.000000e+00  0.000000e+00  0.000000e+00 ... -8.225771e-06 -2.169341e-05
  0.000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
3it [00:01,  2.32it/s]/tmp/ipykernel_7233/

25it [00:11,  2.15it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.00086298 -0.00094738
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
26it [00:11,  2.18it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[-1.2401143e-06  8.8041925e-06  9.2246346e-06 ...  7.4464035e-05
  9.7246913e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
27it [00:12,  2.25it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
28it [00:12,  2.22it/s]/tmp/ipykernel_723

50it [00:25,  1.77it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 2.0218108e-08 -2.4148026e-08  2.6732474e-08 ...  0.0000000e+00
  0.0000000e+00  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
51it [00:25,  1.78it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[-1.3278010e-07  3.9838207e-07 -7.9129450e-07 ...  1.4835584e-04
  9.5790761e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
52it [00:26,  1.67it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[-4.8623036e-07  6.6287242e-07 -8.5870334e-07 ... -5.5028562e-07
 -2.0030720e-06  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.fe

74it [00:41,  1.57it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[-8.1307697e-11  1.5734267e-10 -2.7132227e-10 ... -8.1893653e-05
 -9.9478086e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
75it [00:42,  1.58it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 0.000000e+00  0.000000e+00  0.000000e+00 ...  1.817678e-06 -5.629299e-07
  0.000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
76it [00:42,  1.56it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  3.0983097e-07
 -3.0439924e-07  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.

98it [00:53,  2.12it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 2.4228058e-05  2.2877797e-05  1.4505918e-05 ... -9.0958474e-06
 -8.9365367e-06  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
99it [00:53,  2.13it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[-1.6768606e-06 -1.3229732e-05 -9.4000035e-07 ... -8.8240867e-03
 -9.4295088e-03  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
100it [00:54,  2.08it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  6.6827610e-09
 -5.2103193e-09  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.f

...

171it [01:26,  2.25it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[-2.3759505e-05 -2.3684905e-05 -1.8489012e-05 ...  0.0000000e+00
  0.0000000e+00  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
172it [01:27,  2.21it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  2.3229782e-06
 -1.5600431e-06  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
173it [01:27,  2.18it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[4.252465e-05 7.511085e-05 6.768461e-05 ... 0.000000e+00 0.000000e+00
 0.000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.mel

196it [01:39,  1.81it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[-6.1997180e-06 -1.2222978e-05 -1.0545186e-05 ... -8.7107495e-05
 -1.4605728e-04  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
197it [01:40,  1.79it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 2.4978577e-09 -3.7696610e-09  5.2002083e-09 ...  3.8778231e-05
  3.2255204e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
198it [01:41,  1.68it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 0.000000e+00  0.000000e+00  0.000000e+00 ...  3.180566e-07 -2.832796e-07
  0.000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.featu

220it [01:52,  1.74it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[4.5238266e-05 6.3187872e-05 4.4782333e-05 ... 4.4660683e-05 6.2248182e-05
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
221it [01:53,  1.84it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  4.4518949e-09
 -1.4735161e-07  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
222it [01:53,  1.91it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[-3.1969157e-05  1.7736702e-05 -2.1742735e-05 ...  3.7487505e-05
  5.1390383e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.featu

244it [02:04,  1.94it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 0.000000e+00  0.000000e+00  0.000000e+00 ... -5.896941e-06 -3.698753e-05
  0.000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
245it [02:05,  2.00it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[-1.3347903e-06  2.6771686e-06 -1.2112470e-05 ... -1.4741488e-05
 -1.9867637e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
246it [02:05,  2.00it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 1.8286231e-04  3.1932074e-04  2.9782587e-04 ... -6.3038483e-06
  4.0714381e-06  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.featu

268it [02:16,  1.94it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 1.7171951e-05  3.9974515e-05  1.0753969e-05 ... -9.3529279e-06
 -8.2556808e-06  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
269it [02:17,  1.93it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[ 2.1275762e-05  2.8636508e-05 -3.7183549e-06 ...  2.1056267e-05
  8.1753551e-06  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(x, sr=sample_rate).T,axis=0)
270it [02:17,  1.97it/s]/tmp/ipykernel_7233/840728918.py:25: FutureWarning: Pass y=[2.3644008e-05 5.3299722e-05 4.1212716e-05 ... 5.8258028e-04 5.9082516e-04
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.featu

In [23]:
test_data.tail()

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,178,179,label,name
path,,,,,,,,,,,,,,,,,,,,,
RAVDESS/03-01-06-02-02-01-24.wav,-455.938660,6.955958,-27.052181,-9.363136,-12.501428,-12.745587,-19.336010,-14.905254,-13.156981,3.679700,...,0.003510,0.003252,0.003069,0.003089,0.001303,0.000382,3.240545e-05,6.975525e-07,female_fearful,03-01-06-02-02-01-24.wav
RAVDESS/03-01-08-02-02-02-17.wav,-469.776917,35.147579,2.828279,9.550099,-2.859639,5.900487,-2.742471,3.543220,-6.534330,2.183620,...,0.003212,0.002654,0.002572,0.003130,0.002189,0.000616,8.004962e-05,2.321812e-06,male_surprised,03-01-08-02-02-02-17.wav
RAVDESS/03-01-06-02-01-01-17.wav,-399.929260,31.463903,-6.314829,8.518395,2.319131,4.880717,-0.558570,-1.241705,-4.612333,2.763378,...,0.004573,0.006313,0.004370,0.003574,0.002029,0.000578,5.818321e-05,6.409617e-07,male_fearful,03-01-06-02-01-01-17.wav
RAVDESS/03-01-08-02-02-01-14.wav,-598.394409,33.847713,-10.830214,0.428908,-13.015498,-8.765131,-11.363347,-10.742599,-6.281651,-5.791598,...,0.001468,0.000941,0.001579,0.002203,0.001139,0.000204,1.408182e-05,2.535875e-07,female_surprised,03-01-08-02-02-01-14.wav
RAVDESS/03-01-03-01-01-01-17.wav,-611.179810,63.210636,4.484718,15.666046,10.019296,9.433527,2.350507,-4.195100,-5.128372,3.888745,...,0.000023,0.000022,0.000020,0.000022,0.000014,0.000003,3.185981e-07,5.255963e-09,male_happy,03-01-03-01-01-01-17.wav


In [24]:
train_data.to_csv("waves/train_wave_features.csv", index=False)
test_data.to_csv("waves/test_wave_features.csv", index=False)